In [33]:
from pyspark.sql import  SQLContext
sqlContext=SQLContext(sc)

In [34]:
SAT_FN = 'SAT_Results.csv'
HSD_FN = 'DOE_High_School_Directory_2014-2015.csv'

In [35]:
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load(HSD_FN)

In [36]:
df.show()

+-------------------+--------------------+----+-------------+------------+----------+--------------+--------------+-----------------+-----------------+----+------+----------------------+----+----------+----+-------+--------------+-----------+-----------+------------------+------------------+----------------+-------------------------+-----------------+-----------------------+--------------------------+----------------+-----------------+----------------+-------------+-----------+----------------+----------------+----------------+-----------------+-----------------+-----------------+-------------+-----------+-----------+----------+--------+-----------+------------+--------------------------------+---------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+
|                dbn|         school_name|boro|building_code|phone_number|fax_number|grade_span_min|grade_span_max|expgrade_span_min|expgrade_span_max| bus|sub

In [37]:
sat = sc.textFile(SAT_FN, use_unicode=False).cache()
list(enumerate(sat.first().split(',')))

[(0, 'DBN'),
 (1, 'SCHOOL NAME'),
 (2, 'Num of SAT Test Takers'),
 (3, 'SAT Critical Reading Avg. Score'),
 (4, 'SAT Math Avg. Score'),
 (5, 'SAT Writing Avg. Score')]

In [38]:
def extractScores(index, lines):
    import csv
    if index==0:
        lines.next()
    reader = csv.reader(lines)
    for row in reader:
        if row[2]!='s':
            (dbn, takers, score) = (row[0], int(row[2]), int(row[4]))
            yield (dbn, (score*takers, takers))
        
satScores = sat.mapPartitionsWithIndex(extractScores)

In [39]:
satScores.take(20)

[('02M047', (6400, 16)),
 ('21K410', (207575, 475)),
 ('30Q301', (43120, 98)),
 ('17K382', (22066, 59)),
 ('18K637', (13335, 35)),
 ('32K403', (18300, 50)),
 ('09X365', (18306, 54)),
 ('11X270', (22064, 56)),
 ('05M367', (12078, 33)),
 ('14K404', (24276, 68)),
 ('30Q575', (66420, 135)),
 ('13K336', (3366, 9)),
 ('04M635', (17712, 48)),
 ('24Q264', (40406, 89)),
 ('17K408', (19494, 57)),
 ('19K618', (22260, 60)),
 ('27Q309', (13644, 36)),
 ('32K552', (24388, 67)),
 ('13K499', (26208, 72)),
 ('07X600', (30400, 76))]

In [40]:
schools = sc.textFile(HSD_FN, use_unicode=False).cache()
list(enumerate(schools.first().split(',')))

[(0, 'dbn'),
 (1, 'school_name'),
 (2, 'boro'),
 (3, 'building_code'),
 (4, 'phone_number'),
 (5, 'fax_number'),
 (6, 'grade_span_min'),
 (7, 'grade_span_max'),
 (8, 'expgrade_span_min'),
 (9, 'expgrade_span_max'),
 (10, 'bus'),
 (11, 'subway'),
 (12, 'primary_address_line_1'),
 (13, 'city'),
 (14, 'state_code'),
 (15, 'zip'),
 (16, 'website'),
 (17, 'total_students'),
 (18, 'campus_name'),
 (19, 'school_type'),
 (20, 'overview_paragraph'),
 (21, 'program_highlights'),
 (22, 'language_classes'),
 (23, 'advancedplacement_courses'),
 (24, 'online_ap_courses'),
 (25, 'online_language_courses'),
 (26, 'extracurricular_activities'),
 (27, 'psal_sports_boys'),
 (28, 'psal_sports_girls'),
 (29, 'psal_sports_coed'),
 (30, 'school_sports'),
 (31, 'partner_cbo'),
 (32, 'partner_hospital'),
 (33, 'partner_highered'),
 (34, 'partner_cultural'),
 (35, 'partner_nonprofit'),
 (36, 'partner_corporate'),
 (37, 'partner_financial'),
 (38, 'partner_other'),
 (39, 'addtl_info1'),
 (40, 'addtl_info2'),
 (4

In [41]:
def extractSchools(index, lines):
    import csv
    if index==0:
        lines.next()
    reader = csv.reader(lines)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn, boro, total_students) = (row[0], row[2], int(row[17]))
            if total_students>500:
                yield (dbn, boro)
largeSchools = schools.mapPartitionsWithIndex(extractSchools)

In [42]:
largeSchools.take(10)

[('01M450', 'Manhattan'),
 ('01M539', 'Manhattan'),
 ('01M696', 'Manhattan'),
 ('02M374', 'Manhattan'),
 ('02M400', 'Manhattan'),
 ('02M408', 'Manhattan'),
 ('02M412', 'Manhattan'),
 ('02M413', 'Manhattan'),
 ('02M416', 'Manhattan'),
 ('02M418', 'Manhattan')]

In [43]:
satScores.take(10)

[('02M047', (6400, 16)),
 ('21K410', (207575, 475)),
 ('30Q301', (43120, 98)),
 ('17K382', (22066, 59)),
 ('18K637', (13335, 35)),
 ('32K403', (18300, 50)),
 ('09X365', (18306, 54)),
 ('11X270', (22064, 56)),
 ('05M367', (12078, 33)),
 ('14K404', (24276, 68))]

In [44]:
scores = largeSchools.join(satScores).values()

In [45]:
scores.take(10)

[('Bronx', (23069, 59)),
 ('Staten Island', (52216, 107)),
 ('Bronx', (16317, 49)),
 ('Brooklyn', (33235, 85)),
 ('Bronx', (24570, 65)),
 ('Manhattan', (24948, 66)),
 ('Brooklyn', (17920, 56)),
 ('Bronx', (37120, 80)),
 ('Queens', (180978, 278)),
 ('Queens', (503426, 934))]

In [46]:
largeSchools.join(satScores) \
            .values().collect()

[('Bronx', (23069, 59)),
 ('Staten Island', (52216, 107)),
 ('Bronx', (16317, 49)),
 ('Brooklyn', (33235, 85)),
 ('Bronx', (24570, 65)),
 ('Manhattan', (24948, 66)),
 ('Brooklyn', (17920, 56)),
 ('Bronx', (37120, 80)),
 ('Queens', (180978, 278)),
 ('Queens', (503426, 934)),
 ('Manhattan', (35784, 84)),
 ('Brooklyn', (36822, 102)),
 ('Brooklyn', (48470, 131)),
 ('Brooklyn', (246050, 518)),
 ('Brooklyn', (35992, 88)),
 ('Bronx', (502928, 731)),
 ('Manhattan', (79632, 144)),
 ('Queens', (86304, 174)),
 ('Queens', (153027, 347)),
 ('Bronx', (18084, 44)),
 ('Brooklyn', (59736, 131)),
 ('Bronx', (45288, 111)),
 ('Manhattan', (11776, 32)),
 ('Queens', (75950, 175)),
 ('Brooklyn', (87320, 185)),
 ('Staten Island', (170867, 391)),
 ('Queens', (35760, 80)),
 ('Queens', (164827, 403)),
 ('Queens', (37471, 101)),
 ('Manhattan', (27650, 70)),
 ('Brooklyn', (39330, 95)),
 ('Manhattan', (38232, 81)),
 ('Queens', (163216, 404)),
 ('Bronx', (28116, 66)),
 ('Brooklyn', (214776, 456)),
 ('Bronx', (15552,

In [47]:
largeSchools.join(satScores) \
            .values() \
            .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])).collect()

[('Bronx', (1619364, 3444)),
 ('Manhattan', (3206992, 6228)),
 ('Brooklyn', (4544126, 9322)),
 ('Staten Island', (1406967, 2944)),
 ('Queens', (5190534, 10942))]

In [48]:
largeSchools.join(satScores) \
            .values() \
            .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
            .mapValues(lambda x: x[0]/x[1]) \
            .collect()

[('Bronx', 470),
 ('Manhattan', 514),
 ('Brooklyn', 487),
 ('Staten Island', 477),
 ('Queens', 474)]

In [49]:
largeSchools.join(satScores) \
            .values() \
            .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
            .mapValues(lambda x: x[0]/x[1]) \
            .sortBy(lambda x: -x[1]) \
            .collect()

[('Manhattan', 514),
 ('Brooklyn', 487),
 ('Staten Island', 477),
 ('Queens', 474),
 ('Bronx', 470)]

In [50]:
from pyspark.sql import Row

In [51]:
satRows = satScores.map(lambda x: Row(dbn=x[0], sum_scores=x[1][0], num_takers=x[1][1]))
satTable = sqlContext.createDataFrame(satRows)
satTable.show()

+------+----------+----------+
|   dbn|num_takers|sum_scores|
+------+----------+----------+
|02M047|        16|      6400|
|21K410|       475|    207575|
|30Q301|        98|     43120|
|17K382|        59|     22066|
|18K637|        35|     13335|
|32K403|        50|     18300|
|09X365|        54|     18306|
|11X270|        56|     22064|
|05M367|        33|     12078|
|14K404|        68|     24276|
|30Q575|       135|     66420|
|13K336|         9|      3366|
|04M635|        48|     17712|
|24Q264|        89|     40406|
|17K408|        57|     19494|
|19K618|        60|     22260|
|27Q309|        36|     13644|
|32K552|        67|     24388|
|13K499|        72|     26208|
|07X600|        76|     30400|
+------+----------+----------+
only showing top 20 rows



In [52]:
schoolRows = largeSchools.map(lambda x: Row(dbn=x[0], boro=x[1]))
schoolTable = sqlContext.createDataFrame(schoolRows)
schoolTable.show()

+---------+------+
|     boro|   dbn|
+---------+------+
|Manhattan|01M450|
|Manhattan|01M539|
|Manhattan|01M696|
|Manhattan|02M374|
|Manhattan|02M400|
|Manhattan|02M408|
|Manhattan|02M412|
|Manhattan|02M413|
|Manhattan|02M416|
|Manhattan|02M418|
|Manhattan|02M420|
|Manhattan|02M425|
|Manhattan|02M475|
|Manhattan|02M489|
|Manhattan|02M519|
|Manhattan|02M520|
|Manhattan|02M529|
|Manhattan|02M542|
|Manhattan|02M580|
|Manhattan|02M600|
+---------+------+
only showing top 20 rows



In [53]:
df = schoolTable.join(satTable, on="dbn", how="inner") \
                .groupBy("boro") \
                .sum("sum_scores", "num_takers")

In [54]:
df.show()

+-------------+---------------+---------------+
|         boro|sum(sum_scores)|sum(num_takers)|
+-------------+---------------+---------------+
|       Queens|        5190534|          10942|
|     Brooklyn|        4544126|           9322|
|Staten Island|        1406967|           2944|
|        Bronx|        1619364|           3444|
|    Manhattan|        3206992|           6228|
+-------------+---------------+---------------+



In [55]:
df.withColumn("avg", (df[1]/df[2])) \
          .select('boro', 'avg') \
          .show()

+-------------+------------------+
|         boro|               avg|
+-------------+------------------+
|       Queens| 474.3679400475233|
|     Brooklyn|487.46256168204246|
|Staten Island| 477.9099864130435|
|        Bronx|  470.198606271777|
|    Manhattan| 514.9312780989081|
+-------------+------------------+



In [56]:
def extractSchools(index, lines):
    import csv
    import operator
    if index==0:
        lines.next()
    reader = csv.reader(lines)
    for row in reader:
        if len(row)==58 and row[17].isdigit():
            (dbn, bus, subway) = (row[0], row[10], row[11])
            bus = bus.split(', ')
            subway = reduce(operator.add, map(lambda x: x.split(' to ')[0].split(', '), subway.split('; ')))
            yield (dbn, tuple(bus+subway))
transports = schools.mapPartitionsWithIndex(extractSchools)

In [63]:
transports.take(10)

[('01M292',
  ('B39',
   'M14A',
   'M14D',
   'M15',
   'M15-SBS',
   'M21',
   'M22',
   'M9',
   'B',
   'D',
   'F',
   'J',
   'M',
   'Z')),
 ('01M448', ('M14A', 'M14D', 'M15', 'M21', 'M22', 'M9', 'F', 'J', 'M', 'Z')),
 ('01M450',
  ('M101',
   'M102',
   'M103',
   'M14A',
   'M14D',
   'M15',
   'M15-SBS',
   'M2',
   'M23',
   'M3',
   'M8',
   'M9',
   '6',
   'L')),
 ('01M509',
  ('B39',
   'M103',
   'M14A',
   'M14D',
   'M15',
   'M15-SBS',
   'M21',
   'M22',
   'M8',
   'M9',
   'B',
   'D',
   'F',
   'J',
   'M',
   'Z')),
 ('01M539',
  ('B39', 'M14A', 'M14D', 'M21', 'M22', 'M8', 'M9', 'F', 'J', 'M', 'Z')),
 ('01M696', ('M14A', 'M14D', 'M21', 'M22', 'M9', 'N/A')),
 ('02M047',
  ('M101',
   'M102',
   'M14A',
   'M14D',
   'M15',
   'M15-SBS',
   'M2',
   'M23',
   'M34A-SBS',
   'M5',
   'M9',
   '4',
   '5',
   'Q',
   '6',
   'N',
   'R',
   'L')),
 ('02M135',
  ('M10',
   'M104',
   'M11',
   'M20',
   'M31',
   'M34A-SBS',
   'M42',
   'M50',
   'M57',
   'M7',
  

In [72]:
import itertools
transports.join(satScores) \
          .flatMap(lambda x: itertools.product(x[1][0], [x[1][1]])) \
          .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1])) \
          .mapValues(lambda x: x[0]/x[1]) \
          .sortBy(lambda x: -x[1]) \
          .take(50)

[('S1115', 612),
 ('M79', 594),
 ('Q42', 582),
 ('M22', 574),
 ('Bx3', 571),
 ('B52', 560),
 ('B63', 557),
 ('B69', 548),
 ('B54', 543),
 ('B25', 541),
 ('M20', 540),
 ('M9', 539),
 ('M86', 538),
 ('B65', 538),
 ('B45', 534),
 ('Bx10', 534),
 ('Bx26', 533),
 ('B103', 531),
 ('Q64', 529),
 ('Bx22', 525),
 ('M72', 523),
 ('B41', 520),
 ('B38', 520),
 ('M5', 520),
 ('Q35', 519),
 ('M66', 518),
 ('B62', 513),
 ('3', 513),
 ('A', 510),
 ('C', 510),
 ('Q88', 508),
 ('R', 508),
 ('Q84', 507),
 ('Q30', 507),
 ('Q20A', 505),
 ('S79-SBS', 505),
 ('Q31', 504),
 ('G', 503),
 ('B11', 503),
 ('D', 502),
 ('E', 501),
 ('1', 499),
 ('SIR', 498),
 ('M35', 496),
 ('4', 495),
 ('M10', 495),
 ('Q17', 495),
 ('N', 493),
 ('Q28', 492),
 ('Q13', 492)]

In [73]:
def extractSchools(lines):
    for row in lines:
        if row[17].isdigit():
            (dbn, boro, total_students) = (row[0], row[2], int(row[17]))
            if total_students>500:
                yield (dbn, boro)

from csv_parser import csvRDD
largeSchools1 = csvRDD(schools)

ImportError: No module named csv_parser

In [74]:
largeSchools1.take(10)

NameError: name 'largeSchools1' is not defined